# Fusion

In [ ]:

import pywt
import cv2
import numpy as np
from PIL import Image


class dwt_fusion:
  # This function does the coefficient fusing according to the fusion method
  def fuseCoeff(self, cooef1, cooef2, fusion_method):
    if (fusion_method == 'mean'):
        cooef = (cooef1 + cooef2) / 2
    elif (fusion_method == 'min'):
        cooef = np.minimum(cooef1,cooef2)
    elif (fusion_method == 'max'):
        cooef = np.maximum(cooef1,cooef2)
    else:
        cooef = []
    return cooef


  def fusion(self, fusion_method, img_path1, img_path2, wavelet):

    I1 = cv2.imread(img_path1,1)
    I2 = cv2.imread(img_path2,1)

    channel1 = I1[:,:,0]
    channel2 = I1[:,:,1]
    channel3 = I1[:,:,2]
    first = np.dstack((channel1,channel2,channel3))

## For the first channel
    coef1 = pywt.wavedec2(I1[:,:,0], wavelet)
    coef2 = pywt.wavedec2(I2[:,:,0], wavelet)

    fused = []

    for i in range(len(coef1) - 1):
      if(i == 0):
        fused.append(self.fuseCoeff(coef1[0], coef2[0], fusion_method))

      else:
        c10 = self.fuseCoeff(coef1[i][0], coef2[i][0], fusion_method)
        c20 = self.fuseCoeff(coef1[i][1], coef2[i][1], fusion_method)
        c30 = self.fuseCoeff(coef1[i][2], coef2[i][2], fusion_method)
        fused.append((c10,c20,c30))

    fusedImage0 = pywt.waverec2(fused, wavelet)
    fusedImage0 = np.multiply(np.divide(fusedImage0 - np.min(fusedImage0),(np.max(fusedImage0) - np.min(fusedImage0))),255)
    fusedImage0 = fusedImage0.astype(np.uint8)


## For the second channel
    coef3 = pywt.wavedec2(I1[:,:,1], wavelet)
    coef4 = pywt.wavedec2(I2[:,:,1], wavelet)

    fused = []

    for i in range(len(coef3) - 1):
      if(i == 0):
        fused.append(self.fuseCoeff(coef3[0], coef4[0], fusion_method))

      else:
        c11 = self.fuseCoeff(coef3[i][0], coef4[i][0], fusion_method)
        c21 = self.fuseCoeff(coef3[i][1], coef4[i][1], fusion_method)
        c31 = self.fuseCoeff(coef3[i][2], coef4[i][2], fusion_method)
        fused.append((c11,c21,c31))

    fusedImage1 = pywt.waverec2(fused, wavelet)
    fusedImage1 = np.multiply(np.divide(fusedImage1 - np.min(fusedImage1),(np.max(fusedImage1) - np.min(fusedImage1))),255)
    fusedImage1 = fusedImage1.astype(np.uint8)


## For the third channel
    coef5 = pywt.wavedec2(I1[:,:,2], wavelet)
    coef6 = pywt.wavedec2(I2[:,:,2], wavelet)

    fused = []

    for i in range(len(coef5) - 1):
      if(i == 0):
        fused.append(self.fuseCoeff(coef5[0], coef6[0], fusion_method))

      else:
        c12 = self.fuseCoeff(coef5[i][0], coef6[i][0], fusion_method)
        c22 = self.fuseCoeff(coef5[i][1], coef6[i][1], fusion_method)
        c32 = self.fuseCoeff(coef5[i][2], coef6[i][2], fusion_method)
        fused.append((c12,c22,c32))

    fusedImage2 = pywt.waverec2(fused, wavelet)
    fusedImage2 = np.multiply(np.divide(fusedImage2 - np.min(fusedImage2),(np.max(fusedImage2) - np.min(fusedImage2))),255)
    fusedImage2 = fusedImage2.astype(np.uint8)


    channel11=fusedImage0[:,:]
    channel12=fusedImage1[:,:]
    channel13=fusedImage2[:,:]

    final = np.dstack((channel11,channel12,channel13))


    return(final)

In [ ]:
def resize(img1, img2):
  path = img2
  img1 = Image.open(img1)
  w1, h1 = img1.size
  img2 = Image.open(img2)
  img2 = img2.resize((w1,h1))
  img2.save(path)

In [ ]:
import fusionmodule as fuseMod
from PIL import Image, ImageStat
fuse = fuseMod.dwt_fusion()
resize('/content/dehazed_test_images/dehazed_100001.png','/content/illum/contrastCorrected10226.png')
img = fuse.fusion('mean', '/content/dehazed_test_images/dehazed_100001.png', '/content/illum/contrastCorrected10226.png', 'db4')
cv2.imwrite('FusedImage100001.png',img)

True